In [1]:
import pandas as pd
from openpyxl import load_workbook
import os
import numpy as np

In [2]:
data_path = '..\\datasets'
files = os.listdir(data_path)
files.remove('URL descripcion reto.txt')
files.remove('leeme.txt')
dfs = {}

def open_file(file_name):
    file_path = os.path.join(data_path, file_name)

    # Cargar todas las hojas con pandas.read_excel(sheet_name=None)
    all_sheets = pd.read_excel(file_path, sheet_name=None)

    # Guardar cada hoja en dfs con un nombre único combinando el archivo y la hoja
    for sheet_name, data in all_sheets.items():
        # Crear un nombre único para el DataFrame usando el nombre del archivo y la hoja
        df_name = f"{file_name.replace('.xlsx', '')}_{sheet_name}"
        dfs[df_name] = data


# Abrir todos los archivos y leer todas las hojas
for file in files:
    if file.endswith('.xlsx'):  # Asegurarse de que solo se procesen archivos Excel
        open_file(file)
    if file.endswith('.XLSX'):
        open_file(file)


In [3]:
print(dfs.keys())

dict_keys(['Biorreactor 13169_WData', 'Biorreactor 13169_Datos', 'Biorreactor 13170_WData', 'Biorreactor 13170_Datos', 'Biorreactor 13171_WData', 'Biorreactor 13171_Datos', 'Biorreactor 13172_WData', 'Biorreactor 13172_Datos', 'Biorreactor 14614_Datos', 'Biorreactor 14615_WData', 'Biorreactor 14615_Datos', 'Biorreactor 14616_WData', 'Biorreactor 14616_Datos', 'Biorreactor 14617_WData', 'Biorreactor 14617_Datos', 'Biorreactor 14618_WData', 'Biorreactor 14618_Datos', 'Centrífuga 12912_WData', 'Centrífuga 12912_Datos', 'Centrífuga 14246_WData', 'Centrífuga 14246_Datos', 'Centrífuga 17825_WData', 'Centrífuga 17825_Datos', 'Cinéticos IPC_Inóculos', 'Cinéticos IPC_Cultivos finales', 'Cinéticos IPC_Centrifugación', 'Fases producción v02_Preinóculo', 'Fases producción v02_Inóculo', 'Fases producción v02_Cultivo final', 'Fases producción v03 Test_Cultivo final', 'Fases producción v03_Preinóculo', 'Fases producción v03_Inóculo', 'Fases producción v03_Cultivo final', 'Fases producción_test v02_Cu

In [4]:
# Acceder a los DataFrames

df_preinoculo = dfs['Fases producción v03_Preinóculo']
df_preinoculo.columns = df_preinoculo.iloc[0]
df_preinoculo = df_preinoculo.drop(0)
df_inoculo = dfs['Fases producción v03_Inóculo']
df_cultivo_final = dfs['Fases producción v03 Test_Cultivo final']

# Unir los tres DataFrames por la columna 'LOTE'
df_of_123456 = dfs['OF 123456 v03_Sheet1']
df_of_123456 = df_of_123456.rename(columns={df_of_123456.columns[3]: 'LOTE'})
df_cineticos = dfs['Cinéticos IPC_Inóculos']
df_cineticos = df_cineticos.rename(columns={df_cineticos.columns[0]: 'LOTE'})
df_cineticos = df_cineticos[['LOTE', 'Turbidez', 'Viabilidad']].groupby('LOTE',as_index=False).mean()
df_of_123456['LOTE'] = df_of_123456['LOTE'].astype(str).str.replace('/', '').astype(int)
# Unir los tres DataFrames por la columna 'LOTE'
merged_df = df_preinoculo.merge(df_inoculo, on='LOTE', suffixes=('_preinoculo', '_inoculo'))
merged_df = merged_df.merge(df_of_123456[['LOTE', 'Cantidad entregada']], on='LOTE', suffixes=('', '_OF_123456'))

merged_df = merged_df.merge(df_cineticos, on='LOTE', suffixes=('', 'cineticos'))
# Unir los tres DataFrames por la columna 'LOTE'
merged_df = merged_df.merge(df_inoculo, on='LOTE', suffixes=('', '_inoculo'))
merged_df = merged_df.merge(df_cultivo_final, on='LOTE', how='right', suffixes=('', '_cultivo_final'))

C:\Users\jaime\AppData\Local\Temp\ipykernel_17724\4164120474.py:23: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  merged_df = merged_df.merge(df_cultivo_final, on='LOTE', how='right', suffixes=('', '_cultivo_final'))


In [5]:
merged_df.shape

(55, 43)

In [6]:

merged_df['ID Centrífuga'] = merged_df['ID Centrífuga'].astype(int)
merged_df['ID Bioreactor'] = merged_df['ID Bioreactor'].astype(int)
mode_value = merged_df['ID bioreactor'].mode().iloc[0]  # Get the first mode
merged_df['ID bioreactor'].fillna(mode_value, inplace=True)
merged_df['ID bioreactor'].replace([float('inf'), -float('inf')], mode_value, inplace=True)
merged_df['ID bioreactor'] = merged_df['ID bioreactor'].astype(int)

In [7]:
def load_centrifuga_data(id_centrifuga):
    file_name = f'Centrífuga {id_centrifuga}_Datos'
    try:
        # Intentar cargar el DataFrame de centrífuga
        centrifuga_df = dfs[file_name]
        return centrifuga_df
    except KeyError:
        # Si el archivo no existe, retornar un DataFrame vacío o NaN
        return pd.DataFrame()  # o simplemente return np.nan si prefieres


def calculate_mean_for_lote(c, fecha_inicio, fecha_fin, id_centrifuga):
    centrifuga_df = load_centrifuga_data(id_centrifuga)
    
    if centrifuga_df.empty:
        return np.nan  # Si no hay datos, devolver NaN
    mask = (pd.to_datetime(centrifuga_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & (pd.to_datetime(centrifuga_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = centrifuga_df[mask]
    return filtered_data[f'{id_centrifuga}_D01916047.PV'].mean()

merged_df['media_PV'] = merged_df.apply(
    lambda row: calculate_mean_for_lote(row['LOTE'], row['Fecha/hora inicio'], row['Fecha/hora fin'], row['ID Centrífuga']),
    axis=1
)

In [8]:
def calculate_mean_for_lote_temp(lote, fecha_inicio, fecha_fin):
    temp_df = dfs['Temperaturas y humedades_Datos']
    if temp_df.empty:
        return np.nan  # Si no hay datos, devolver NaN
    mask = (pd.to_datetime(temp_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & (pd.to_datetime(temp_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = temp_df[mask]
    return filtered_data[['07781_TI1501.PV','07781_MI1501.PV']].mean()


merged_df[['temperature_preinoculo','humedad_preinoculo']] = merged_df.apply(
    lambda row: calculate_mean_for_lote_temp(row['LOTE'],row['Fecha/hora inicio_preinoculo'], row['Fecha/hora fin_preinoculo']), axis = 1)

In [9]:

import pandas as pd
import numpy as np

# Function to load bioreactor data based on the given ID
def load_bioreactor_data(id_bioreactor):
    file_name = f'Biorreactor {id_bioreactor}_Datos'
    try:
        # Attempt to load the DataFrame of the bioreactor
        bioreactor_df = dfs[file_name]
        return bioreactor_df
    except KeyError:
        # If the file does not exist, return an empty DataFrame
        return pd.DataFrame()  # Alternatively, return np.nan if preferred

# Function to calculate the mean for the specified lot and bioreactor
def calculate_mean_for_lote2(lote, fecha_inicio, fecha_fin, id_bioreactor):
    bioreactor_df = load_bioreactor_data(id_bioreactor)
    
    if bioreactor_df.empty:
        return np.nan, np.nan, np.nan  # Return NaN for all means if there's no data

    # Filter the data based on the specified date range
    mask = (pd.to_datetime(bioreactor_df['DateTime']) >= pd.to_datetime(fecha_inicio)) & \
           (pd.to_datetime(bioreactor_df['DateTime']) <= pd.to_datetime(fecha_fin))
    filtered_data = bioreactor_df[mask]
    
    # Calculate means for the specified columns
    mean_temp = filtered_data[f'{id_bioreactor}_FERM0101.Temperatura_PV'].mean() if not filtered_data.empty else np.nan
    mean_ph = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_pH_PV'].mean() if not filtered_data.empty else np.nan
    mean_do = filtered_data[f'{id_bioreactor}_FERM0101.Single_Use_DO_PV'].mean() if not filtered_data.empty else np.nan
    
    return mean_temp, mean_ph, mean_do

# Apply the function to each row of merged_df and store the results in new columns
merged_df[['media_temp_bioreactor', 'media_ph_biorreactor', 'media_PO_biorreactor']] = merged_df.apply(
    lambda row: calculate_mean_for_lote2(row['LOTE'], row['Fecha/hora inicio_inoculo'], row['Fecha/hora fin_inoculo'], row['ID bioreactor']),
    axis=1,
    result_type='expand'  # This allows unpacking into multiple columns
)

In [10]:
merged_df = merged_df.fillna("NA")
output_file_path = '../dataset_test_new.csv'  # Cambia esto a la ruta donde quieras guardar el archivo
merged_df.to_csv(output_file_path, index=False)

In [11]:
merged_df

,LOTE,Fecha/hora inicio_preinoculo,Fecha/hora fin_preinoculo,línea 1,línea 2,línea 3,línea 1,línea 2,línea 3,línea 1,...,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2,media_PV,temperature_preinoculo,humedad_preinoculo,media_temp_bioreactor,media_ph_biorreactor,media_PO_biorreactor
0,24054.0,2024-04-14 00:30:00,2024-04-15 07:17:00,5.504,5.376,5.376,27.12,25.68,29.52,1,...,27.84,23.96,NA,NA,0.692889,20.265144,47.023889,29.523502,5.889634,23.939733
1,24055.0,2024-04-11 00:30:00,2024-04-12 07:14:00,5.368,5.416,5.464,26.16,25.52,25.04,1,...,30.96,23.16,NA,NA,0.65288,20.153662,45.463467,29.478246,5.909516,24.440044
2,24056.0,2024-04-11 00:30:00,2024-04-12 07:14:00,5.368,5.416,5.464,26.16,25.52,25.04,1,...,29.52,28.88,NA,NA,0.439311,20.153662,45.463467,29.478246,5.909516,24.440044
3,24057.0,2024-04-14 00:30:00,2024-04-15 07:17:00,5.504,5.376,5.376,27.12,25.68,29.52,1,...,31.04,25.32,NA,NA,0.598215,20.265144,47.023889,29.523502,5.889634,23.939733
4,24058.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,26.08,20.36,NA,NA,NA,NA,NA,NA,NA,NA
5,24059.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,26.88,23.64,NA,NA,NA,NA,NA,NA,NA,NA
6,24060.0,2024-04-18 00:30:00,2024-04-19 07:46:00,5.384,5.344,5.448,25.44,26,24.72,1,...,32.56,17.92,NA,NA,0.351869,20.169626,49.159645,29.480435,5.881805,24.754048
7,24061.0,2024-04-18 00:30:00,2024-04-19 07:46:00,5.384,5.344,5.448,25.44,26,24.72,1,...,27.76,14.72,NA,NA,1.157978,20.169626,49.159645,29.480435,5.881805,24.754048
8,24062.0,NA,NA,NA,NA,NA,NA,NA,NA,NA,...,27.52,24.64,NA,NA,NA,NA,NA,NA,NA,NA
9,24063.0,2024-04-21 00:30:00,2024-04-22 07:19:00,5.36,5.376,NA,25.84,27.84,NA,1,...,25.92,18.28,NA,NA,0.629609,20.165213,44.743246,29.46004,5.866393,24.954162
